In [2]:
import numpy as np
import math
import pandas as pd

np.seterr(all="raise")  # Error on overflow

# INPUTS
LOAD_DEAD = 8000  # kN
LOAD_LIVE = 2500  # kN
BEARINGPRESSURE_ALLOWABLE = 150  # kPa
FTG_COVER = 0.060  # m
COLUMN_WIDTH = 0.500  # m

# Costs
COST_CONC = 400 / 25  # per m^3 per mpa
COST_REO = 3.70  # $/kg

# Constants
REO_DENSITY = 7850  # kg/m3


class FoundationSizes:
    def __init__(self):
        """Initialize the FoundationSizes class and calculate all properties."""
        self.sizes = self.generate_foundation_sizes()
        self.initialize_attributes()
        self.calculate_all_properties()

    def generate_foundation_sizes(self):
        """Generate the foundation sizes based on initial parameters."""
        FTG_LEN_MIN = (
            math.ceil(
                math.sqrt((LOAD_DEAD + LOAD_LIVE) / BEARINGPRESSURE_ALLOWABLE) / 0.05
            )
            * 0.05
        )
        FTG_LEN_MAX = FTG_LEN_MIN + 1
        FTG_LENS = np.round(
            np.arange(FTG_LEN_MIN, FTG_LEN_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_DPTH_MIN = (
            math.ceil(
                (
                    4
                    * math.sqrt(3570)
                    * math.sqrt(
                        (4760 * COLUMN_WIDTH**2) / 3
                        + (1 + (3 * BEARINGPRESSURE_ALLOWABLE) / 19040)
                        * (LOAD_DEAD + LOAD_LIVE)
                    )
                    - (9520 + 3 * BEARINGPRESSURE_ALLOWABLE) * COLUMN_WIDTH
                )
                / (19040 + 3 * BEARINGPRESSURE_ALLOWABLE)
                / 0.05
            )
            * 0.05
        )
        FTG_DPTH_MAX = FTG_DPTH_MIN * 2
        FTG_DPTHS = np.round(
            np.arange(FTG_DPTH_MIN, FTG_DPTH_MAX + 0.001, 0.05, dtype=np.float32), 2
        )

        FTG_CONC_STRENGTHS = np.array([20, 25, 32, 40, 50, 65], dtype=np.float32)

        FTG_REO_SIZES = np.round(
            np.array(
                [0.01, 0.012, 0.016, 0.02, 0.024, 0.028, 0.032, 0.036, 0.04],
                dtype=np.float32,
            ),
            3,
        )

        FTG_REO_CTS = np.unique(
            np.round(
                np.concatenate(
                    [
                        np.arange(0.1, 0.301, 0.025, dtype=np.float32),
                        np.arange(0.08, 0.301, 0.02, dtype=np.float32),
                    ]
                ),
                3,
            )
        )

        foundation_sizes = np.array(
            np.meshgrid(
                FTG_LENS, FTG_DPTHS, FTG_CONC_STRENGTHS, FTG_REO_SIZES, FTG_REO_CTS
            )
        ).T.reshape(-1, 5)

        # Instantiate the full array with additional empty columns
        sizes = np.zeros((foundation_sizes.shape[0], 33), dtype=np.float32)
        sizes[:, :5] = foundation_sizes
        return sizes

    def initialize_attributes(self):
        """Initialize attributes for foundation properties."""
        attributes = [
            "FtgLength",
            "FtgDepth",
            "fc",
            "ReoSize",
            "ReoCts",
            "BPmax",
            "BP_Ratio",
            "Pult",
            "Dom",
            "BPult",
            "CLR",
            "VPult",
            "fVP",
            "VP_Ratio",
            "dv",
            "VLult",
            "kvo",
            "kv",
            "ks",
            "fVuc",
            "VL_Ratio",
            "Mult",
            "Astshr",
            "Astmin",
            "Ast",
            "alpha",
            "gamma",
            "ku",
            "phi",
            "Astact",
            "fMuo",
            "M_Ratio",
            "cost",
        ]

        for i, attr in enumerate(attributes):
            setattr(self, attr, self.sizes[:, i])

    def calculate_all_properties(self):
        """Calculate all foundation properties."""
        calc_methods = [
            self.calc_BPmax,
            self.calc_Pult,
            self.calc_ratio_bearing,
            self.calc_Dom,
            self.calc_BPult,
            self.calc_CLR,
            self.calc_VPult,
            self.calc_fVP,
            self.calc_VP_Ratio,
            self.calc_dv,
            self.calc_VLult,
            self.calc_kvo,
            self.calc_kv,
            self.calc_ks,
            self.calc_fVuc,
            self.calc_VL_Ratio,
            self.calc_Mult,
            self.calc_Astshr,
            self.calc_Astmin,
            self.calc_Ast,
            self.calc_alpha,
            self.calc_gamma,
            self.calc_ku,
            self.calc_phi,
            self.calc_Astact,
            self.calc_fMuo,
            self.calc_M_Ratio,
            self.calc_cost,
        ]

        for method in calc_methods:
            method()

    def calc_BPmax(self):
        """Calculate maximum bearing pressure."""
        self.BPmax[:] = (
            6 * self.FtgDepth * self.FtgLength**2 + LOAD_LIVE + LOAD_DEAD
        ) / (self.FtgLength**2)

    def calc_ratio_bearing(self):
        """Calculate bearing pressure ratio."""
        self.BP_Ratio[:] = BEARINGPRESSURE_ALLOWABLE / self.BPmax

    def calc_Pult(self):
        """Calculate ultimate load."""
        self.Pult[:] = (
            1.2 * (6 * self.FtgDepth * self.FtgLength**2 + LOAD_DEAD) + 1.5 * LOAD_LIVE
        )

    def calc_Dom(self):
        """Calculate depth of the moment arm."""
        self.Dom[:] = self.FtgDepth - FTG_COVER - self.ReoSize / 2

    def calc_BPult(self):
        """Calculate ultimate bearing pressure."""
        self.BPult[:] = self.Pult / self.FtgLength**2

    def calc_CLR(self):
        """Calculate column load ratio."""
        self.CLR[:] = self.BPult * (COLUMN_WIDTH + self.Dom) ** 2

    def calc_VPult(self):
        """Calculate ultimate vertical load."""
        self.VPult[:] = self.Pult - self.CLR

    def calc_fVP(self):
        """Calculate vertical load capacity"""
        self.fVP[:] = (
            952
            * (self.ReoSize - self.Dom)
            * (self.ReoSize - self.Dom - COLUMN_WIDTH)
            * np.sqrt(self.fc)
        )

    def calc_VP_Ratio(self):
        """Calculate vertical load capacity ratio"""
        self.VP_Ratio[:] = self.VPult / self.fVP

    def calc_dv(self):
        """Calculate vertical load displacement"""
        self.dv[:] = np.maximum(0.9 * self.Dom, 0.72 * self.FtgDepth)

    def calc_VLult(self):
        """Calculate ultimate vertical load"""
        self.VLult[:] = (
            -0.5 * self.BPult * (COLUMN_WIDTH + 2 * self.dv - self.FtgLength)
        )

    def calc_kvo(self):
        self.kvo[:] = 2 / (10 + 13 * self.dv)

    def calc_kv(self):
        self.kv[:] = np.minimum(self.kvo, 0.15)

    def calc_ks(self):
        self.ks[:] = np.maximum(0.5, (10 / 7) * (1 - self.FtgDepth))

    def calc_fVuc(self):
        self.fVuc[:] = (
            700 * self.FtgDepth * self.dv * np.sqrt(self.fc) * self.ks * self.kv
        )

    def calc_VL_Ratio(self):
        self.VL_Ratio[:] = self.VLult / self.fVuc

    # Mult per metre
    def calc_Mult(self):
        self.Mult[:] = (
            self.BPult * (7 * COLUMN_WIDTH - 10 * self.FtgLength) ** 2
        ) / 800

    def calc_Astshr(self):
        self.Astshr[:] = (5 * self.Mult) / (2 * self.Dom)

    def calc_Astmin(self):
        self.Astmin[:] = (228 * self.FtgDepth**2 * np.sqrt(self.fc)) / self.Dom

    def calc_Ast(self):
        self.Ast[:] = np.maximum(self.Astmin, self.Astshr)

    def calc_alpha(self):
        self.alpha[:] = 0.85 - 0.0015 * self.fc

    def calc_gamma(self):
        self.gamma[:] = 0.97 - 0.0025 * self.fc

    def calc_ku(self):
        self.ku[:] = self.Ast / (
            2000 * self.alpha * self.Dom * self.fc * self.gamma * self.FtgLength
        )

    def calc_phi(self):
        self.phi[:] = np.minimum(0.85, np.maximum(0.65, 1.24 - 1.08333 * self.ku))

    def calc_Astact(self):
        self.Astact[:] = (250000 * self.ReoSize**2 * np.pi) / self.ReoCts

    # phi Muo per metre
    def calc_fMuo(self):
        self.fMuo[:] = (self.Astact * self.Dom * self.phi) / 2 - (
            self.Astact**2 * self.phi
        ) / (8000 * self.alpha * self.fc * self.FtgLength)

    def calc_M_Ratio(self):
        self.M_Ratio[:] = self.Mult / self.fMuo

    def calc_cost(self):
        self.cost[:] = (
            self.Astact / 1000000 * self.FtgLength * REO_DENSITY * 2 * COST_REO
            + self.FtgLength**2 * self.FtgDepth * COST_CONC * self.fc
        )

    def remove_fails(self):
        self.sizes = self.sizes[
            (self.M_Ratio <= 1) & (self.VL_Ratio <= 1) & (self.VP_Ratio <= 1) & (self.Astact >= self.Ast)
        ]

    def sort_by_cost(self):
        """Sort the foundation sizes by cost from lowest to highest."""
        cost_index = 32
        sorted_indices = np.argsort(self.sizes[:, cost_index])
        self.sizes = self.sizes[sorted_indices]

    def print_raw_array(self, num_rows=5):
        """Print the raw array of foundation sizes."""
        print(self.sizes[0, :])

    def print_foundation_sizes(self, num_rows=6):
        """Print a sample of the foundation sizes."""
        column_headers = [
            attr
            for attr in self.__dict__
            if isinstance(getattr(self, attr), np.ndarray)
        ][1:]
        pd.set_option("display.float_format", lambda x: "%.3f" % x)
        num_columns = len(column_headers)

        total_rows = self.sizes.shape[0]
        step = max(1, total_rows // (num_rows - 1))

        selected_rows = np.arange(0, total_rows, step)
        if len(selected_rows) < num_rows:
            selected_rows = np.append(selected_rows, total_rows - 1)
        elif len(selected_rows) > num_rows:
            selected_rows = selected_rows[:num_rows]

        df = pd.DataFrame(
            self.sizes[selected_rows, :num_columns],
            columns=column_headers[:num_columns],
            index=selected_rows,
        )
        print(df.to_string(header=True))


# Create an instance of the FoundationSizes class
foundation = FoundationSizes()
foundation.remove_fails()
foundation.sort_by_cost()
foundation.print_foundation_sizes()
foundation.sizes.shape

       FtgLength  FtgDepth     fc  ReoSize  ReoCts   BPmax  BP_Ratio      Pult   Dom   BPult      CLR     VPult       fVP  VP_Ratio    dv   VLult   kvo    kv    ks    fVuc  VL_Ratio     Mult   Astshr   Astmin      Ast  alpha  gamma    ku   phi    Astact      fMuo  M_Ratio       cost
0          8.400     2.100 32.000    0.024   0.160 161.410     0.929 14416.867 2.028 204.321 1305.769 13111.098 27023.646     0.485 1.825 434.141 0.059 0.059 0.500 450.005     0.965 1655.062 2040.263 2804.661 2804.661  0.802  0.890 0.004 0.850  2827.433  2433.025    0.680  77245.766
4536       9.100     2.050 40.000    0.032   0.160 139.096     1.078 14572.275 1.974 175.972 1077.070 13495.205 28553.662     0.473 1.777 444.049 0.060 0.060 0.500 487.190     0.911 1684.111 2132.866 3069.908 3069.908  0.790  0.870 0.003 0.850  5026.549  4207.688    0.400 111303.859
9072       8.950     1.950 50.000    0.032   0.240 142.782     1.051 14474.640 1.874 180.701 1018.411 13456.229 29040.127     0.463 1.687 458.693 0.

(22680, 33)